In [1]:
import clingo

from typing import Sequence

In [2]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' ', report_answersets=True):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            if report_answersets:
                print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [3]:
def check(programs: Sequence[str], consistency_checks:str, grounding_context=None):
    print("Run without checks:")
    wo_checks = solve(programs, grounding_context, report_answersets=False)
    print("Run with checks:")
    w_checks = solve((*programs, consistency_checks), grounding_context, report_answersets=False)
    return len(wo_checks) == len(w_checks)

In [4]:
domain = """
contract("Instanz Tauschvertrag").

component("Lieferung Ration").
component("Rückabwicklung Ration").

component("Lieferung Brett").
component("Nachfrist Lieferung Brett").
component("Lieferung Brett cont.").

component("Lieferung Brett Austauschanspruch").
component("Lieferung Brett Differenzanspruch").

componentType("Sachleistung").
componentType("Nachfrist").
componentType("Geldleistung").
componentType("Austauschanspruch").
componentType("Differenzanspruch").

componentState("Initialisiert").
componentState("Gewährleistung").
componentState("Abgeschlossen").
componentState("Initialisiert [GLE]").
componentState("Gewährleistung [GLE]").
componentState("Abgeschlossen [GLE]").
componentState("Unmöglichkeit (Schuldner)").
componentState("Unmöglichkeit (Schuldner) [GLE]").
% componentState("Unmöglichkeit (Gläubiger)").
% componentState("Unmöglichkeit (Gläubiger) [GLE]").
% componentState("Unmöglichkeit (Zufall)").
% componentState("Unmöglichkeit (Zufall) [GLE]").
% componentState("Verzug (Schuldner)").
% componentState("Verzug (Schuldner) [GLE]").
% componentState("Verzug (Schuldner, Vertrag besteht)").
% componentState("Verzug (Schuldner, Vertrag besteht) [GLE]").
% componentState("Verzug (Schuldner, Vom Vertrag zurückgetreten)").
% componentState("Verzug (Schuldner, Vom Vertrag zurückgetreten) [GLE]").
% componentState("Verzug (Gläubiger)").
% componentState("Verzug (Gläubiger) [GLE]").

transitionAction("GLE").
transitionAction("LE").
transitionAction("GWA").
transitionAction("UNM(S)").

state_transAction_state("Initialisiert","GLE","Initialisiert [GLE]").

state_transAction_state("Initialisiert","UNM(S)","Unmöglichkeit (Schuldner)").

state_transAction_state("Initialisiert","GLE","Unmöglichkeit (Schuldner) [GLE]").
state_transAction_state("Initialisiert","UNM(S)","Unmöglichkeit (Schuldner) [GLE]").

state_transAction_state("Initialisiert","LE","Gewährleistung").

state_transAction_state("Initialisiert [GLE]","LE","Gewährleistung [GLE]").

state_transAction_state("Initialisiert [GLE]","UNM(S)","Unmöglichkeit (Schuldner) [GLE]").

state_transAction_state("Gewährleistung","GLE","Gewährleistung [GLE]").

state_transAction_state("Gewährleistung","GWA","Abgeschlossen").

state_transAction_state("Gewährleistung","GLE","Abgeschlossen [GLE]").
state_transAction_state("Gewährleistung","GWA","Abgeschlossen [GLE]").

state_transAction_state("Gewährleistung [GLE]","GWA","Abgeschlossen [GLE]").

state_transAction_state("Abgeschlossen","GLE","Abgeschlossen [GLE]").

state_transAction_state("Unmöglichkeit (Schuldner)", "GLE", "Unmöglichkeit (Schuldner) [GLE]").

resource("Holz").
resource("Nahrung").
resource("Brett").
resource("Ration").
resource("Gemälde").
resource("Geld").

resourceType("Gattung").
resourceType("Spezies").
resourceType("Geld").

agent(bob).
agent(woody).

event(destroy(Resource)) :- resource(Resource).

action(gather("Holz")).
action(gather("Nahrung")).
action(process_to("Holz", "Brett")).
action(process_to("Nahrung", "Ration")).
action(give_to_because(Resource, Agent, Component)) :- agent(Agent), resource(Resource), component(Component).
action(accept_from_because(Resource, Agent, Component)) :- agent(Agent), resource(Resource), component(Component).

action(communicate_to_because("Am Vertrag festhalten", Agent, Component)) :- agent(Agent), component(Component).
action(communicate_to_because("Vom Vertrag zurücktreten", Agent, Component)) :- agent(Agent), component(Component).

time(0..10).

"""
solve([domain]);

Answer 1: { action(gather("Holz")) action(gather("Nahrung")) action(process_to("Holz","Brett")) action(process_to("Nahrung","Ration")) action(accept_from_because("Brett",bob,"Lieferung Brett")) action(accept_from_because("Brett",bob,"Lieferung Brett Austauschanspruch")) action(accept_from_because("Brett",bob,"Lieferung Brett Differenzanspruch")) action(accept_from_because("Brett",bob,"Lieferung Brett cont.")) action(accept_from_because("Brett",bob,"Lieferung Ration")) action(accept_from_because("Brett",bob,"Nachfrist Lieferung Brett")) action(accept_from_because("Brett",bob,"Rückabwicklung Ration")) action(accept_from_because("Brett",woody,"Lieferung Brett")) action(accept_from_because("Brett",woody,"Lieferung Brett Austauschanspruch")) action(accept_from_because("Brett",woody,"Lieferung Brett Differenzanspruch")) action(accept_from_because("Brett",woody,"Lieferung Brett cont.")) action(accept_from_because("Brett",woody,"Lieferung Ration")) action(accept_from_because("Brett",woody,"Nac

In [5]:
consistency_checks = """
:- deo_at(component_state(Component, State1), T), deo_at(component_state(Component, State2), T), State1 != State2.
:- deo_at(component_state(Component, State), T), deo_at(component_state(Component, SuccState), T+1), State != SuccState, not state_transAction_state(State,_, SuccState).
:- state_transAction_state(State, TransAction, SuccState), not componentState(State).
:- state_transAction_state(State, TransAction, SuccState), not transitionAction(TransAction).
:- state_transAction_state(State, TransAction, SuccState), not componentState(SuccState).
"""

In [6]:
contract = """

contract_component("Instanz Tauschvertrag", "Lieferung Ration").
contract_component("Instanz Tauschvertrag", "Lieferung Brett").

component_type("Lieferung Ration", "Sachleistung").
component_provider("Lieferung Ration", bob).
component_beneficiary("Lieferung Ration", woody).
component_item("Lieferung Ration", "Ration").

component_type("Lieferung Brett", "Sachleistung").
component_provider("Lieferung Brett", woody).
component_beneficiary("Lieferung Brett", bob).
component_item("Lieferung Brett", "Brett").

deo_at(component_state("Lieferung Ration", "Initialisiert"), 0).
deo_at(component_state("Lieferung Brett", "Initialisiert"), 0).

"""
solve([contract]);

Answer 1: { component_beneficiary("Lieferung Brett",bob) component_beneficiary("Lieferung Ration",woody) component_item("Lieferung Brett","Brett") component_item("Lieferung Ration","Ration") component_provider("Lieferung Brett",woody) component_provider("Lieferung Ration",bob) component_type("Lieferung Brett","Sachleistung") component_type("Lieferung Ration","Sachleistung") contract_component("Instanz Tauschvertrag","Lieferung Brett") contract_component("Instanz Tauschvertrag","Lieferung Ration") deo_at(component_state("Lieferung Brett","Initialisiert"),0) deo_at(component_state("Lieferung Ration","Initialisiert"),0) }
SAT 1


In [7]:
instance = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(woody, destroy("Brett")), 3).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Ration")), 4).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Ration")), 4).

"""
solve([instance]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Ration")),4) occ_at(agent_does(woody,destroy("Brett")),3) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Brett")),1) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Ration")),4) }
SAT 1


In [8]:
rules = """

component_dual(Component, Dual) :-
 component(Component), component(Dual),
 contract(Contract),
 Component != Dual,
 contract_component(Contract, Component), contract_component(Contract, Dual).

% If the transition event does not occur then there is no transition
-transComp_state_at(Component, SuccState, T) :-
  component(Component), componentState(SuccState), time(T),
  componentState(State), transitionAction(TransAction),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, TransAction, SuccState),
  not trans_comp_at(TransAction, Component, T).

% If two states have the same successor state and both states can be reached they will not transition
-transComp_state_at(Component, State1, T) :-
  component(Component), componentState(State1), time(T),
  componentState(State), componentState(State2), transitionAction(TransAction1), transitionAction(TransAction2),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, TransAction1, State1),
  state_transAction_state(State, TransAction2, State2),
  state_transAction_state(State1, TransAction2, SuccState),
  state_transAction_state(State2, TransAction1, SuccState),
  trans_comp_at(TransAction1, Component, T),
  trans_comp_at(TransAction2, Component, T).

% If a transition is not impossible then it will be executed
transComp_state_at(Component, SuccState, T) :-
  component(Component), componentState(SuccState), time(T),
  componentState(State),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, _, SuccState),
  not -transComp_state_at(Component, SuccState, T).

% INERTIA
deo_at(component_state(Component, State), T+1) :-
  component(Component), componentState(State), time(T+1),
  time(T),
  deo_at(component_state(Component, State), T),
  not transComp_state_at(Component, _, T).

% TRANSITION
deo_at(component_state(Component, SuccState), T+1) :-
  component(Component), componentState(SuccState), time(T+1),
  time(T), componentState(State),
  deo_at(component_state(Component, State), T),
  transComp_state_at(Component, SuccState, T).

% Physical Exchange leads to Deontic Transaction
rul_comp_at(transaction_from_to(Item, Provider, Beneficiary), Component, T) :-
  resource(Item), agent(Provider), agent(Beneficiary), component(Component), time(T),
  component_provider(Component, Provider),
  component_beneficiary(Component, Beneficiary),
  component_item(Component, Item),
  occ_at(agent_does(Provider, give_to_because(Item, Beneficiary, Component)),     T),
  occ_at(agent_does(Beneficiary, accept_from_because(Item, Provider, Component)), T).

% Deontic Transaction is a LE Transition Event
trans_comp_at("LE", Component, T) :-
  component(Component), time(T),
  rul_comp_at(transaction_from_to(_, _, _), Component, T).

trans_comp_at("GLE", Component, T) :-
  component(Component), time(T),
  component(Dual),
  component_dual(Component, Dual),
  trans_comp_at("LE", Dual, T).

%%% Unmöglichkeit %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% If an agent destroys an component item, the component became impossible
rul_comp_at(impossibility(Agent), Component, T) :-
  agent(Agent), component(Component), time(T),
  resource(Item),
  component_item(Component, Item),
  occ_at(agent_does(Agent, destroy(Item)), T).

%%% Unmöglichkeit Schuldner %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% If the provider caused the impossiblity the component transitions to the Unmöglichkeit (Schuldner) state
trans_comp_at("UNM(S)", Component, T) :-
  component(Component), time(T),
  agent(Provider),
  component_provider(Component, Provider),
  rul_comp_at(impossibility(Provider), Component, T).

%%% Unmöglichkeit Schuldner [GLE] %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% No additional rules

"""
solve([rules]);

Answer 1: {  }
SAT 1


<block>:4:2-22: info: atom does not occur in any rule head:
  component(Component)

<block>:4:24-39: info: atom does not occur in any rule head:
  component(Dual)

<block>:5:2-20: info: atom does not occur in any rule head:
  contract(Contract)

<block>:7:2-41: info: atom does not occur in any rule head:
  contract_component(Contract,Component)

<block>:7:43-77: info: atom does not occur in any rule head:
  contract_component(Contract,Dual)

<block>:11:3-23: info: atom does not occur in any rule head:
  component(Component)

<block>:11:25-50: info: atom does not occur in any rule head:
  componentState(SuccState)

<block>:11:52-59: info: atom does not occur in any rule head:
  time(T)

<block>:12:3-24: info: atom does not occur in any rule head:
  componentState(State)

<block>:12:26-55: info: atom does not occur in any rule head:
  transitionAction(TransAction)

<block>:14:3-57: info: atom does not occur in any rule head:
  state_transAction_state(State,TransAction,SuccState)

<block>

In [9]:
solve([
    domain,
    rules,
    contract,
    instance,
    '#show occ_at/2.',
    '#show obs_at/2.',
    '#show deo_at/2.',
    '#show rul_comp_at/3.'
    #'#show trans_comp_at/3.',
    #'#show transComp_state_at/3.',
    #'#show -transComp_state_at/3.',
    #'#show state_transAction_state/3.',
], sep='\n');

Answer 1: {
deo_at(component_state("Lieferung Brett","Initialisiert"),0)
deo_at(component_state("Lieferung Brett","Initialisiert"),1)
deo_at(component_state("Lieferung Brett","Initialisiert"),2)
deo_at(component_state("Lieferung Brett","Initialisiert"),3)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner)"),4)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner) [GLE]"),5)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner) [GLE]"),6)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner) [GLE]"),7)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner) [GLE]"),8)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner) [GLE]"),9)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner) [GLE]"),10)
deo_at(component_state("Lieferung Ration","Gewährleistung"),5)
deo_at(component_state("Lieferung Ration","Gewährleistung"),6)
deo_at(component_state("Lieferung Ration","Gewährleistung"),7)
deo_at(

<block>:1:1-16: info: no atoms over signature occur in program:
  obs_at/2



In [10]:
check([
    domain,
    rules,
    contract,
    instance,], consistency_checks)

Run without checks:
SAT 1
Run with checks:
SAT 1


True